In [77]:
import os

In [4]:
!pip install --quiet --upgrade datasets langchain langchain-community langchain-chroma sentence-transformers umap-learn matplotlib scikit-learn
!pip install --upgrade PyMuPDF
!pip install huggingface_hub
!pip install pandas requests beautifulsoup4 PyMuPDF datasets \
    langchain langchain-community langchain-chroma sentence-transformers \
    scikit-learn matplotlib torch

In [ ]:
# import sys
# print(sys.executable)

In [78]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import fitz  # PyMuPDF
from datasets import load_dataset
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import torch
from sklearn.metrics.pairwise import cosine_similarity

## Load Data

In [215]:
df_disease = pd.read_csv("/kaggle/input/scraped-output-metadata-new-csv/scraped_output_metadata_new.csv")
df_family = pd.read_csv("/kaggle/input/scraped-output/scraped_output_famresource_metadata.csv")
df_housing = pd.read_csv("/kaggle/input/scraped-output/scraped_output_housing_metadata_new.csv")
df_meal = pd.read_csv("/kaggle/input/scraped-output/scraped_output_meal_metadata_new.csv")
df_alldisease = pd.read_csv("/kaggle/input/scraped-output-alldisease-other/scraped_output_alldisease_metadata_new.csv")
df_other = pd.read_csv("/kaggle/input/scraped-output-alldisease-other/scraped_output_otherrelevant_metadata_new.csv")

In [220]:
df = pd.concat([df_disease, df_family, df_housing, df_meal, df_alldisease, df_other], axis = 0)

In [217]:
pd.set_option("display.max_colwidth", 1000)
# df = pd.read_csv("/kaggle/input/scraped-output-metadata-new-csv/scraped_output_metadata_new.csv")


In [218]:
import os
os.getcwd()

'/kaggle/working'

In [221]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5779 entries, 0 to 3563
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  5779 non-null   int64 
 1   text        5778 non-null   object
 2   metadata    5779 non-null   object
dtypes: int64(1), object(2)
memory usage: 180.6+ KB


In [ ]:
# df = df.rename(columns= {'scraped_Content': 'scraped_content'})

In [ ]:
# df_filtered = df.drop(index = [4, 5, 14, 15, 24, 25])

In [ ]:
# df_filtered_reset_index = df_filtered.reset_index()
# df_filtered_reset_index = df_filtered_reset_index.drop(columns = 'index')
# df_filtered_reset_index.info()

In [ ]:
# df_filtered_reset_index.sample(2)

In [ ]:
#import pandas as pd

# Load the uploaded CSV
#file_path = "/content/scraped_output_metadata.csv"
#df = pd.read_csv(file_path)

# Remove duplicate rows based on the "text" column (keeping the first occurrence)
#df_cleaned = df.drop_duplicates(subset=["text"], keep="first").reset_index(drop=True)

# Shift the numbering if there's an index-like column (e.g., auto increment)
# For demonstration, create a new sequential index column starting from 0
#df_cleaned.insert(0, "new_index", range(len(df_cleaned)))

# Save cleaned file
#output_path = "/content/scraped_output_metadata_cleaned.csv"
#df_cleaned.to_csv(output_path, index=False)

#output_path


'/content/scraped_output_metadata_cleaned.csv'

In [ ]:
#df = df_cleaned

In [ ]:
pd.set_option("display.max_colwidth", 500)

In [ ]:
df.head()

,Unnamed: 0,text,metadata
0,0,"Your child’s treatment for cancer depends on the type of cancer they have and how advanced it is. Treatments may include chemotherapy, radiation therapy, surgery, targeted therapy, immunotherapy, and stem cell (bone marrow) transplant. Find trusted resources you can understand to help you as you make decisions.\nAllogeneic Stem Cell (Bone Marrow) Transplant\nAutologous Stem Cell (Bone Marrow) Transplant\nBrachytherapy\nCAR T-Cell Therapy\nChemotherapy\nImmunotherapy\nProton Beam Radiation\nR...","{'source_url': 'https://together.stjude.org/en-us/conditions/cancers/cancer-treatments.html', 'retrieved_at': '2025-09-29T18:09:04.907923', 'title': 'Cancer Treatments - Together by St. Jude™', 'section': 'Learn about treatments for childhood and adolescent cancers', 'super_section': None}"
1,1,"TreatmentsTreatments for childhood illnesses include medicine, surgery, radiation, stem cell transplant, immunotherapy, or targeted therapy. Learn about these treatments.\nTreatments for childhood illnesses include medicine, surgery, radiation, stem cell transplant, immunotherapy, or targeted therapy. Learn about these treatments.\nHow Treatments May Affect YouKnowing the facts about treatment and side effects can help you be informed about your care. Get the facts about how treatment works ...","{'source_url': 'https://together.stjude.org/en-us/conditions/cancers/cancer-treatments.html', 'retrieved_at': '2025-09-29T18:09:04.907923', 'title': 'Cancer Treatments - Together by St. Jude™', 'section': 'Featured resources', 'super_section': None}"
2,2,A new radiation treatment may treat brain tumors with fewer side effects so patients can function better after treatment.,"{'source_url': 'https://together.stjude.org/en-us/conditions/cancers/cancer-treatments.html', 'retrieved_at': '2025-09-29T18:09:04.907923', 'title': 'Cancer Treatments - Together by St. Jude™', 'section': 'Research Spotlight', 'super_section': None}"
3,3,"Managing symptoms and side effects is important to your child’s recovery and well-being. Learn more about cancer symptoms and treatment side effects.\nANC and Neutropenia\nAnemia\nAvascular Necrosis (Osteonecrosis)\nBlood Clots\nCentral Venous Line (Catheter) Complications\nChemo Brain\nConstipation\nCytokine Release Syndrome\nDehydration\nDelayed Wound Healing\nDelirium\nDiarrhea\nDysarthria\nExcessive Daytime Sleepiness, Hypersomnia, and Narcolepsy\nFatigue\nFever and Signs of Infection\nG...","{'source_url': 'https://together.stjude.org/en-us/conditions/cancers/cancer-symptoms-side-effects.html', 'retrieved_at': '2025-09-29T18:09:05.128189', 'title': 'Cancer Symptoms and Side Effects - Together by St. Jude™', 'section': 'Learn about managing your child’s symptoms and side effects', 'super_section': None}"
4,4,Coping with Side EffectsSide effects are unintended effects or unwanted symptoms that occur because of medicines or other treatments. Learn how to manage and cope with side effects.\nSide effects are unintended effects or unwanted symptoms that occur because of medicines or other treatments. Learn how to manage and cope with side effects.\nPain ManagementPain management is an important part of care for children with cancer or other illnesses. Learn ways to prevent and manage pain.\nPain mana...,"{'source_url': 'https://together.stjude.org/en-us/conditions/cancers/cancer-symptoms-side-effects.html', 'retrieved_at': '2025-09-29T18:09:05.128189', 'title': 'Cancer Symptoms and Side Effects - Together by St. Jude™', 'section': 'Featured resources', 'super_section': None}"


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1116 entries, 0 to 1115
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  1116 non-null   int64 
 1   text        1116 non-null   object
 2   metadata    1116 non-null   object
dtypes: int64(1), object(2)
memory usage: 26.3+ KB


In [222]:
import ast

df['metadata'] = df['metadata'].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)

In [119]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1551 entries, 0 to 77
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  1551 non-null   int64 
 1   text        1550 non-null   object
 2   metadata    1551 non-null   object
dtypes: int64(1), object(2)
memory usage: 48.5+ KB


In [102]:
df = df.iloc[:, 1:]

In [223]:
df = df.dropna()

In [224]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5778 entries, 0 to 3563
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  5778 non-null   int64 
 1   text        5778 non-null   object
 2   metadata    5778 non-null   object
dtypes: int64(1), object(2)
memory usage: 180.6+ KB


## Document Indexing

### Chunking

In [225]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from typing import Iterable, List

def chunk_text(
    text: str,
    chunk_size: int = 800,
    chunk_overlap: int = 80
) -> List[str]:
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=["\n\n", "\n", ".", " "]
    )
    return splitter.split_text(text)

def build_chunks_dataframe(
    df: pd.DataFrame,
    chunk_size: int = 800,
    chunk_overlap: int = 80
) -> pd.DataFrame:

    records = []

    for source_idx, row in df.iterrows():
        seg_text = row['text']
        seg_metadata = row['metadata']

        seg_chunks = chunk_text(seg_text, chunk_size, chunk_overlap)

        for chunk_idx, chunk in enumerate(seg_chunks):
            meta = seg_metadata.copy()
            meta['source_idx'] = source_idx
            meta['chunk_idx'] = chunk_idx

            records.append({'chunk_text': chunk,
                            'metadata': meta})

    return pd.DataFrame(records)

In [226]:
df_chunks = build_chunks_dataframe(df)

In [227]:
df_chunks

,chunk_text,metadata
0,"Your child’s treatment for cancer depends on the type of cancer they have and how advanced it is. Treatments may include chemotherapy, radiation therapy, surgery, targeted therapy, immunotherapy, and stem cell (bone marrow) transplant. Find trusted resources you can understand to help you as you make decisions.\nAllogeneic Stem Cell (Bone Marrow) Transplant\nAutologous Stem Cell (Bone Marrow) Transplant\nBrachytherapy\nCAR T-Cell Therapy\nChemotherapy\nImmunotherapy\nProton Beam Radiation\nRadiation Therapy\nStem Cell Transplant\nSurface Guided Radiation Therapy (SGRT)\nSurgery\nTargeted Therapy","{'source_url': 'https://together.stjude.org/en-us/conditions/cancers/cancer-treatments.html', 'retrieved_at': '2025-09-29T18:09:04.907923', 'title': 'Cancer Treatments - Together by St. Jude™', 'section': 'Learn about treatments for childhood and adolescent cancers', 'super_section': None, 'source_idx': 0, 'chunk_idx': 0}"
1,"TreatmentsTreatments for childhood illnesses include medicine, surgery, radiation, stem cell transplant, immunotherapy, or targeted therapy. Learn about these treatments.\nTreatments for childhood illnesses include medicine, surgery, radiation, stem cell transplant, immunotherapy, or targeted therapy. Learn about these treatments.\nHow Treatments May Affect YouKnowing the facts about treatment and side effects can help you be informed about your care. Get the facts about how treatment works and how it may affect you now and later in life.\nKnowing the facts about treatment and side effects can help you be informed about your care. Get the facts about how treatment works and how it may affect you now and later in life.","{'source_url': 'https://together.stjude.org/en-us/conditions/cancers/cancer-treatments.html', 'retrieved_at': '2025-09-29T18:09:04.907923', 'title': 'Cancer Treatments - Together by St. Jude™', 'section': 'Featured resources', 'super_section': None, 'source_idx': 1, 'chunk_idx': 0}"
2,"Video LibraryListen to hopeful thoughts, medical explanations, and practical advice from families and care teams facing childhood and adolescent cancer, blood disorders, and other serious illnesses.\nListen to hopeful thoughts, medical explanations, and practical advice from families and care teams facing childhood and adolescent cancer, blood disorders, and other serious illnesses.","{'source_url': 'https://together.stjude.org/en-us/conditions/cancers/cancer-treatments.html', 'retrieved_at': '2025-09-29T18:09:04.907923', 'title': 'Cancer Treatments - Together by St. Jude™', 'section': 'Featured resources', 'super_section': None, 'source_idx': 1, 'chunk_idx': 1}"
3,A new radiation treatment may treat brain tumors with fewer side effects so patients can function better after treatment.,"{'source_url': 'https://together.stjude.org/en-us/conditions/cancers/cancer-treatments.html', 'retrieved_at': '2025-09-29T18:09:04.907923', 'title': 'Cancer Treatments - Together by St. Jude™', 'section': 'Research Spotlight', 'super_section': None, 'source_idx': 2, 'chunk_idx': 0}"
4,"Managing symptoms and side effects is important to your child’s recovery and well-being. Learn more about cancer symptoms and treatment side effects.\nANC and Neutropenia\nAnemia\nAvascular Necrosis (Osteonecrosis)\nBlood Clots\nCentral Venous Line (Catheter) Complications\nChemo Brain\nConstipation\nCytokine Release Syndrome\nDehydration\nDelayed Wound Healing\nDelirium\nDiarrhea\nDysarthria\nExcessive Daytime Sleepiness, Hypersomnia, and Narcolepsy\nFatigue\nFever and Signs of Infection\nGraft Versus Host Disease\nHair Loss\nHearing Loss\nInfertility in Male Cancer and Stem Cell Transplant Patients\nInsomnia\nJaundice\nLoss of Appetite and Weight Loss\nMalignant Hyperthermia\nMouth and Throat Sores\nNausea and Vomiting During Cancer Treatment\nNeuropathic Pain\nPeripheral Neuropathy\nPosterior Fossa Syndrome","{'source_url': 'https://together.stjude.org/en-us/conditions/cancers/cancer-symptoms-side-effects.html', 'retrieved_at

In [228]:
# df['chunk_text'] = df['scraped_content'].apply(lambda x: chunk_text(x, chunk_size = 800, chunk_overlap = 80))

In [229]:
# df_chunks['chunk_count'] = df_chunks['chunk_text'].apply(len)

In [230]:
# from itertools import chain

# all_chunks = list(chain.from_iterable(df_chunks['chunk_text']))

# total_chunks = len(all_chunks)
# total_doc = len(df_chunks)
# print(f"Total Chunk Texts: {total_chunks}")
# print(f"Average Chunk per Document: {(total_chunks / total_doc):.2f}")

In [231]:
all_chunks = df_chunks['chunk_text'].tolist()
all_metas = df_chunks['metadata'].tolist()

In [232]:
len(all_chunks)

6346

In [233]:
all_chunks[:5]

['Your child’s treatment for cancer depends on the type of cancer they have and how advanced it is. Treatments may include chemotherapy, radiation therapy, surgery, targeted therapy, immunotherapy, and stem cell (bone marrow) transplant. Find trusted resources you can understand to help you as you make decisions.\nAllogeneic Stem Cell (Bone Marrow) Transplant\nAutologous Stem Cell (Bone Marrow) Transplant\nBrachytherapy\nCAR T-Cell Therapy\nChemotherapy\nImmunotherapy\nProton Beam Radiation\nRadiation Therapy\nStem Cell Transplant\nSurface Guided Radiation Therapy (SGRT)\nSurgery\nTargeted Therapy',
 'TreatmentsTreatments for childhood illnesses include medicine, surgery, radiation, stem cell transplant, immunotherapy, or targeted therapy. Learn about these treatments.\nTreatments for childhood illnesses include medicine, surgery, radiation, stem cell transplant, immunotherapy, or targeted therapy. Learn about these treatments.\nHow Treatments May Affect YouKnowing the facts about trea

In [234]:
all_metas[:5]

[{'source_url': 'https://together.stjude.org/en-us/conditions/cancers/cancer-treatments.html',
  'retrieved_at': '2025-09-29T18:09:04.907923',
  'title': 'Cancer Treatments   - Together by St. Jude™',
  'section': 'Learn about treatments for childhood and adolescent cancers',
  'super_section': None,
  'source_idx': 0,
  'chunk_idx': 0},
 {'source_url': 'https://together.stjude.org/en-us/conditions/cancers/cancer-treatments.html',
  'retrieved_at': '2025-09-29T18:09:04.907923',
  'title': 'Cancer Treatments   - Together by St. Jude™',
  'section': 'Featured resources',
  'super_section': None,
  'source_idx': 1,
  'chunk_idx': 0},
 {'source_url': 'https://together.stjude.org/en-us/conditions/cancers/cancer-treatments.html',
  'retrieved_at': '2025-09-29T18:09:04.907923',
  'title': 'Cancer Treatments   - Together by St. Jude™',
  'section': 'Featured resources',
  'super_section': None,
  'source_idx': 1,
  'chunk_idx': 1},
 {'source_url': 'https://together.stjude.org/en-us/conditions/

### Embedding

In [235]:
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

print(f"Setting device to '{device}'")

Setting device to 'cuda'


In [236]:
from typing import List
# Use a pre-trained Sentence-BERT model to convert each chunk into a semantic vector
# embedder = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
# embedder = HuggingFaceEmbeddings(model_name="thenlper/gte-small")

def embed_chunk_docs(chunk_docs: List[str],
                     model_name: str = "sentence-transformers/all-MiniLM-L6-v2",
                     normalize: bool = True):
    """
    Embed chunks of documents based on a given model name.
    """

    embedder = HuggingFaceEmbeddings(
    model_name= model_name,
    encode_kwargs={"normalize_embeddings": normalize})

    embeddings = embedder.embed_documents(chunk_docs)

    return embedder, embeddings

In [237]:
# embedder, embeddings = embed_chunk_docs(all_chunks, "intfloat/e5-large-v2", True)
# embedder, embeddings = embed_chunk_docs(all_chunks, "sentence-transformers/all-MiniLM-L6-v2", True)
embedder, embeddings = embed_chunk_docs(all_chunks, "sentence-transformers/paraphrase-MiniLM-L6-v2", True)

In [238]:
len(embeddings)

6346

In [239]:
len(embeddings[0])

384

### Store Embeddings in Chroma DB

In [240]:
# !rm -rf ./chroma_scifact

In [241]:
from langchain_chroma import Chroma
# from langchain.vectorstores import Chroma
from langchain.embeddings.base import Embeddings
import time
from pathlib import Path
from typing import Optional, Union


def store_embed_in_db(texts: List[str],
    metas: List[dict],
    embedder: Optional[Embeddings] = None,
    persist_directory: Optional[Union[str, Path]] = None
) -> Chroma:
    """
    Store text chunks (and their embeddings) in a Chroma vector database.

    You must provide *either* `embeddings` (precomputed) *or* an `embedder`
    (model that implements .embed_documents).

    Args:
        texts: List of text chunks to index.
        embedder: Optional Embeddings object; used to compute embeddings on the fly.
        persist_directory: Directory where the Chroma DB will be saved.
            If None, a timestamped folder will be created.

    Returns:
        A Chroma vector store instance with your texts indexed.
    """

    if embedder is None:
        raise ValueError("You must supply an `embedder`")

    if persist_directory is None:
        ts = int(time.time())
        persist_directory = Path(f"./chroma_scifact_run_{ts}")
    persist_directory = Path(persist_directory)

    all_metas = df_chunks['metadata'].tolist()


    db = Chroma.from_texts(
        texts=texts,
        embedding=embedder,
        metadatas = metas,
        persist_directory=str(persist_directory)
    )

    print("Chunks successfully stored in Chroma vector DB.")

    return db

In [242]:
db = store_embed_in_db(all_chunks, all_metas, embedder = embedder)

Chunks successfully stored in Chroma vector DB.


In [250]:
import pickle
from langchain.vectorstores import Chroma

# Assume vectordb = Chroma(...)
db.persist()  # writes to vectordb._persist_directory

# Save just the persist_directory path
with open("chroma_store.pkl", "wb") as f:
    pickle.dump(db._persist_directory, f)

print("Chroma DB persisted and path saved to pickle.")

AttributeError: 'Chroma' object has no attribute 'persist'

In [243]:
col = db._collection  # the underlying chromadb Collection
data = col.get()

In [244]:
len(data['documents'])

6346

In [245]:
meta_section_set = set()

for meta in all_metas:
    meta_section_set.add(meta.get('section', ''))

In [246]:
meta_section_set

{'Storing essential oils',
 'Risk factors for kidney tumors',
 'Why You Should Get Your Flu Vaccine',
 'Ways to prevent foodborne illness',
 'What is low-grade glioma (LGG)?',
 'Rotavirus',
 'Symptoms of AIHA',
 'Support for new patient families',
 'Symptoms of kidney tumors',
 'What is parenteral nutrition?',
 'Living with hemophilia',
 'Managing Pain Without Medicine - Together by St. Jude™',
 'What is health insurance?',
 'Clinical research volunteers',
 'How is a modified barium swallow test used?',
 'Plasma treatments',
 'Before massage therapy',
 'How to preventCandidainfection',
 'Possible side effects of leukapheresis',
 'Stages of melanoma',
 'As a colleague',
 'Screening and planning',
 'Thursday',
 'Symptoms of hereditary spherocytosis',
 'What are the breast cancer screening guidelines for survivors?',
 'Health Insurance Marketplace',
 'Long tubes',
 'Possible side effects\xa0of the low-dose ACTH stimulation test',
 'Change positions often',
 'NG tube (nasogastric), NJ tube

In [147]:
meta_super_section_set = set()

for meta in all_metas:
    meta_super_section_set.add(meta.get('super_section', ''))

In [148]:
meta_super_section_set

{None}

In [149]:
# all_metas.get('section', '')

## Query-Response Pipeline (RAG)

In [209]:
# Improved retrieval function with query expansion and better ranking
from langchain.vectorstores.base import VectorStoreRetriever
import re
from typing import List, Tuple, Dict, Any

def expand_query(query: str) -> List[str]:
    """Expand query with related terms for better retrieval"""
    expansions = {
        # Medical terms
        "symptoms": ["signs", "indications", "early signs", "clinical features"],
        "treatment": ["therapy", "management", "care", "intervention"],
        "side effects": ["adverse effects", "toxicities", "complications"],
        "diagnosis": ["testing", "screening", "evaluation"],
        "prognosis": ["outlook", "survival", "recovery chances"],
        "clinical trial": ["research study", "trial", "experimental therapy"],
        # Cancer / disease
        "cancer": ["tumor", "carcinoma", "malignancy", "oncology"],
        "leukemia": ["blood cancer", "ALL", "AML"],
        "lymphoma": ["hodgkin", "non-hodgkin"],
        "infection": ["illness", "disease", "condition", "virus", "bacteria"],
        # Resources
        "meal": ["food", "nutrition", "dining", "cafeteria", "diet"],
        "housing": ["lodging", "accommodation", "place to stay", "family housing"],
        "transportation": ["shuttle", "travel", "ride", "commute", "bus service"],
        "financial": ["cost", "expenses", "billing", "assistance", "funding"],
        "support": ["counseling", "resources", "services", "help", "aid"],
        # Hospital / logistics
        "admission": ["application", "check-in", "eligibility", "requirements"],
        "visitor": ["guest", "family member", "parent", "guardian"],
        "appointment": ["clinic visit", "consultation", "checkup", "follow-up"],
    }

    expanded_queries = [query]
    query_lower = query.lower()

    for key, synonyms in expansions.items():
        if key in query_lower:
            for synonym in synonyms:
                expanded_queries.append(query.replace(key, synonym, 1))

    return expanded_queries

def detect_metadata_filter(query: str) -> Optional[Dict[str, Any]]:
    """
    Look for trigger phrases in the query and return an appropriate
    metadata filter for your vector store.
    """
    q = query.lower()
    # 2) detect_metadata_filter() dictionary
    mapping = {
        # --- St. Jude navigation & new patients ---
        "new patient":      {"section": {"$eq": "Information for New Patients | St. Jude Care & Treatment"}},
        "first visit":      {"section": {"$eq": "Information for New Patients | St. Jude Care & Treatment"}},
    
        # --- Housing ---
        "housing":          {"section": {"$eq": "Housing and Patient Services - St. Jude Children’s Research Hospital"}},
        "target house":     {"section": {"$eq": "Housing and Patient Services"}},
        "tri delta":        {"section": {"$eq": "Housing and Patient Services"}},
        "domino's village": {"section": {"$eq": "Domino's Village Menu"}},
    
        # --- Food services / menus ---
        "kay kafe":         {"section": {"$eq": "Kay Kafe Menu"}},
        "kay cafe":         {"section": {"$eq": "Kay Kafe Menu"}},
        "room service":     {"section": {"$eq": "Room Service Menu - St. Jude Children’s Research Hospital"}},
        "isolation":        {"section": {"$eq": "Isolation To-Go Menu - St. Jude Children’s Research Hospital"}},
        "menu":             {"section": {"$eq": "Kay Kafe Menu"}},
    
        # --- Logistics ---
        "parking":          {"section": {"$eq": "Parking and Construction | St. Jude Care & Treatment"}},
        "directions":       {"section": {"$eq": "Parking and Construction | St. Jude Care & Treatment"}},
        "visitor":          {"section": {"$eq": "Visitors"}},
    
        # --- Patient portal & programs ---
        "mychart":          {"section": {"$eq": "St. Jude MyChart"}},
        "child life":       {"section": {"$eq": "Child Life | St. Jude Care & Treatment"}},
        "school program":   {"section": {"$eq": "The St. Jude School Program | St. Jude Care & Treatment"}},
    
        # --- Clinical trials / research ---
        "clinical trial":   {"section": {"$eq": "Clinical Trials | St. Jude Care & Treatment"}},
        "consent":          {"section": {"$eq": "Understanding and signing consent forms"}},
    
        # --- Policies / legal ---
        "hipaa":            {"title":   {"$eq": "Notice of Privacy Practices (HIPAA) - St. Jude Children’s Research Hospital"}},
        "privacy":          {"section": {"$eq": "Privacy & Legal"}},
        "nondiscrimination":{"section": {"$eq": "Discrimination is against the law"}},
    
        # --- Support services / community ---
        "concierge":        {"section": {"$eq": "Best Upon Request Concierge Service\u202f | St. Jude Care & Treatment"}},
        "st. jude voice":   {"section": {"$eq": "St. Jude Voice: Our Virtual Adviser community | St. Jude Care & Treatment"}},
    
        # --- Treatments ---
        "chemotherapy":     {"section": {"$eq": "Chemotherapy"}},
        "radiation":        {"section": {"$eq": "Radiation therapy"}},
        "surgery":          {"section": {"$eq": "Surgery"}},
        "targeted therapy": {"section": {"$eq": "Targeted therapy"}}
}

   

    for trigger, filt in mapping.items():
        if trigger in q:
            return filt
    return None


def retrive_top_k_docs_improved(selected_query: str,
                               db_retriever: VectorStoreRetriever,
                               embedder: Embeddings,
                               k_docs: int = 10,  # Increased from 5
                               lambda_mult: float = 0.3,  # Lower for more similarity focus
                               query_prefix: str = "query: ",
                               doc_prefix: str = "passage: ",
                               use_query_expansion: bool = True):
    """
    Improved retrieval with query expansion and better re-ranking
    """

    if embedder is None:
        raise ValueError("You must supply an `embedder`.")

    # Step 1: Query expansion
    if use_query_expansion:
        expanded_queries = expand_query(selected_query)
    else:
        expanded_queries = [selected_query]

    # Step 2: Auto‐detect a metadata filter
    metadata_filter = detect_metadata_filter(selected_query)

    # Step 3: Retrieve documents for each expanded query with metadata filter
    all_retrieved_docs = []

    for exp_query in expanded_queries:
        formatted_query = f"{query_prefix}{exp_query}"

        # Define retriever method
        retriever = db_retriever.as_retriever(
            search_type="mmr",
            search_kwargs={"k": k_docs,
                           "lambda_mult": lambda_mult,
                          **({"filter": metadata_filter} if metadata_filter else {})}
        )

        # Get relevant documents
        results = retriever.get_relevant_documents(formatted_query)

        # Store documents with their expanded query context
        for doc in results:
            all_retrieved_docs.append((doc.page_content, exp_query))

    # Step 3: Remove duplicates while preserving order
    seen_docs = set()
    unique_docs = []
    for doc_content, exp_query in all_retrieved_docs:
        if doc_content not in seen_docs:
            seen_docs.add(doc_content)
            unique_docs.append((doc_content, exp_query))

    # Step 4: Re-rank all unique documents against original query
    original_formatted_query = f"{query_prefix}{selected_query}"
    query_embed = embedder.embed_query(original_formatted_query)

    # Format documents for embedding
    formatted_retrieved_docs = [f"{doc_prefix}{doc}" for doc, _ in unique_docs]

    # Embed documents
    doc_embeds = embedder.embed_documents(formatted_retrieved_docs)

    # Compute similarity scores
    similarity_scores = cosine_similarity([query_embed], doc_embeds)[0]

    # Step 5: Combine documents with scores and rank
    ranked_docs = []
    for i, (formatted_doc, score) in enumerate(zip(formatted_retrieved_docs, similarity_scores)):
        original_doc, exp_query = unique_docs[i]
        ranked_docs.append((formatted_doc, score, exp_query))

    # Sort by similarity score (descending)
    ranked_docs.sort(key=lambda x: x[1], reverse=True)

    # Step 6: Apply keyword boost for exact matches
    boosted_docs = []
    query_keywords = set(re.findall(r'\b\w+\b', selected_query.lower()))

    for doc, score, exp_query in ranked_docs:
        doc_keywords = set(re.findall(r'\b\w+\b', doc.lower()))
        keyword_overlap = len(query_keywords.intersection(doc_keywords))

        # Boost score based on keyword overlap
        boosted_score = score + (keyword_overlap * 0.05)  # Small boost for keyword matches
        boosted_docs.append((doc, boosted_score, exp_query))

    # Final sort by boosted scores
    boosted_docs.sort(key=lambda x: x[1], reverse=True)

    # Display results
    print("Query:", selected_query)
    print(f"\nExpanded to {len(expanded_queries)} queries")
    print(f"Retrieved {len(unique_docs)} unique documents")
    print(f"\nTop {k_docs} Retrieved Chunks with Scores:\n")

    final_results = []
    for i, (doc, score, exp_query) in enumerate(boosted_docs[:k_docs]):
        print(f"Chunk {i+1} (Score: {score:.4f}, From: '{exp_query}'):")
        print(f"{doc}")
        print(f"{'-'*80}")
        final_results.append((doc, score))

    return boosted_docs[:k_docs]

In [210]:
# Compare original vs improved retrieval with question selection
QUERIES = {"1": "What are brain and spinal cord tumors?",
           "2": "What are the most uncommon tumors?",
           "3": "What is adrenocortical tumor?",
           "4": "What is Family Commons at St. Jude?"}

print("Available queries:")
for key, value in QUERIES.items():
    print(f"{key}: {value}")

query_choice = input("Enter number of your query choice: ")
selected_query = QUERIES.get(query_choice)

if selected_query:
    print(f"\nSelected query: {selected_query}")

    print("\n" + "=" * 50)
    print("IMPROVED RETRIEVAL METHOD")
    print("=" * 50)

    result_improved = retrive_top_k_docs_improved(selected_query,
                       db_retriever = db,
                       embedder = embedder,
                       k_docs = 10,
                       lambda_mult = 0.3,
                       query_prefix = 'query: ',
                       doc_prefix = 'passage: ',
                       use_query_expansion = True)
else:
    print("Invalid choice. Please select 1, 2, or 3.")

Available queries:
1: What are brain and spinal cord tumors?
2: What are the most uncommon tumors?
3: What is adrenocortical tumor?
4: What is Family Commons at St. Jude?


Enter number of your query choice:  3



Selected query: What is adrenocortical tumor?

IMPROVED RETRIEVAL METHOD
Query: What is adrenocortical tumor?

Expanded to 1 queries
Retrieved 10 unique documents

Top 10 Retrieved Chunks with Scores:

Chunk 1 (Score: 0.8285, From: 'What is adrenocortical tumor?'):
passage: International Pediatric Adrenocortical Tumors Registry (IPACTR)
--------------------------------------------------------------------------------
Chunk 2 (Score: 0.7765, From: 'What is adrenocortical tumor?'):
passage: Adrenocortical tumor (ACT) is a rare tumor of theadrenal gland. These tumors can bebenignormalignant. You might also hear ACT called:
Adrenocortical cancer
Cancer of the adrenal cortex
Adrenocortical adenoma
Adrenocortical carcinoma
The body has 2 adrenal glands, 1 on top of each kidney. The adrenal glands produce hormones such as cortisol and aldosterone. These hormones have important jobs such as:
Controlling how the body uses carbs, fats, and proteins for energy
Helping the body cope with stress
Co

In [211]:
from langchain.vectorstores.base import VectorStoreRetriever

def create_rag_prompt(selected_query: str,
                       db_retriever: VectorStoreRetriever,
                       embedder: Embeddings,
                       k_docs: int = 5,
                       lambda_mult: float = 0.3,
                       query_prefix: str = "query: ",
                       doc_prefix: str = "passage: ",
                       use_query_expansion: bool = True):
    """
    Retrieve the top-k most relevant chunks for a given query using
    Maximal Marginal Relevance, then re-rank by cosine similarity.

    Args:
        query: Raw query string.
        db_retriever: The `db.as_retriever(...)` object configured with MMR.
        embedder: An Embeddings instance (implements embed_query & embed_documents).
        k: How many chunks to return.
        lambda_mult: MMR diversity parameter (0→pure similarity, 1→pure diversity).
        query_prefix: Text to prepend before embedding the query.
        doc_prefix: Text to prepend before embedding each chunk.

    Returns:
        A list of (chunk_text, score) sorted descending by score.
    """

    if embedder is None:
        raise ValueError("You must supply an `embedder`.")

    # Step 1: Query expansion
    if use_query_expansion:
        expanded_queries = expand_query(selected_query)
    else:
        expanded_queries = [selected_query]

    # Step 2: Auto‐detect a metadata filter
    metadata_filter = detect_metadata_filter(selected_query)

    # Step 3: Retrieve documents for each expanded query with metadata filter
    all_retrieved_docs = []

    for exp_query in expanded_queries:
        formatted_query = f"{query_prefix}{exp_query}"

        # Define retriever method
        retriever = db_retriever.as_retriever(
            search_type="mmr",
            search_kwargs={"k": k_docs,
                           "lambda_mult": lambda_mult,
                          **({"filter": metadata_filter} if metadata_filter else {})}
        )

        # Get relevant documents
        results = retriever.get_relevant_documents(formatted_query)

        # Store documents with their expanded query context
        for doc in results:
            all_retrieved_docs.append((doc.page_content, exp_query))

    # Step 4: Remove duplicates while preserving order
    seen_docs = set()
    unique_docs = []
    for doc_content, exp_query in all_retrieved_docs:
        if doc_content not in seen_docs:
            seen_docs.add(doc_content)
            unique_docs.append((doc_content, exp_query))

    # Step 5: Re-rank all unique documents against original query
    original_formatted_query = f"{query_prefix}{selected_query}"
    query_embed = embedder.embed_query(original_formatted_query)

    # Format documents for embedding
    formatted_retrieved_docs = [f"{doc_prefix}{doc}" for doc, _ in unique_docs]

    # Embed documents
    doc_embeds = embedder.embed_documents(formatted_retrieved_docs)

    # Compute similarity scores
    similarity_scores = cosine_similarity([query_embed], doc_embeds)[0]

    # Step 6: Combine documents with scores and rank
    ranked_docs = []
    for i, (formatted_doc, score) in enumerate(zip(formatted_retrieved_docs, similarity_scores)):
        original_doc, exp_query = unique_docs[i]
        ranked_docs.append((formatted_doc, score, exp_query))

    # Sort by similarity score (descending)
    ranked_docs.sort(key=lambda x: x[1], reverse=True)

    # Step 7: Apply keyword boost for exact matches
    boosted_docs = []
    query_keywords = set(re.findall(r'\b\w+\b', selected_query.lower()))

    for doc, score, exp_query in ranked_docs:
        doc_keywords = set(re.findall(r'\b\w+\b', doc.lower()))
        keyword_overlap = len(query_keywords.intersection(doc_keywords))

        # Boost score based on keyword overlap
        boosted_score = score + (keyword_overlap * 0.05)  # Small boost for keyword matches
        boosted_docs.append((doc, boosted_score, exp_query))

    # Final sort by boosted scores
    boosted_docs.sort(key=lambda x: x[1], reverse=True)

    # Combine the retrieved context into a single block of text
    context = "\n".join([doc for doc, score, exp_query in boosted_docs])

    # Compose the RAG prompt by combining the context and the user's question
    rag_prompt = f"""
You are a helpful healthcare information assistant for families, patients, and caregivers.
You specialize in providing clear, accurate, and supportive information about childhood diseases,
cancer, and St. Jude Children’s Research Hospital.

When answering:
- Always rely on the retrieved documents (disease info pages, hospital resources, treatment guides).
- Present information in clear, compassionate language that non-experts can understand.
- Never provide personal medical advice, diagnoses, or treatment recommendations.
  Instead, encourage patients to consult their doctors for medical concerns.

Answer structure:
If relevant information is found:
- Summarize the key facts clearly and accurately.
- Organize the response with short paragraphs, bullet points, or sections if helpful.
- Cite or mention the source section in natural language (e.g., “According to St. Jude’s page on brain tumors…”).
- Keep the tone supportive and empathetic.
- Please provide answers in English only

Style:
- Use simple, reassuring language.
- Avoid technical jargon unless the retrieved text explicitly includes it.
- Do not speculate or invent facts.

Remember: Your role is to **inform, not advise**. Always guide families toward
qualified healthcare providers for decisions about diagnosis or treatment.


    [CONTEXT]
    {context}
    """

    # Display the final composed prompt that will be passed to the LLM
    print("=== RAG Augmented Prompt ===")
    print(rag_prompt)

    return rag_prompt


In [213]:
# Define a new query
QUERIES = {"1": "What are the types of common tumors?",
           "2": "What are the most uncommon tumors?",
           "3": "What is adrenocortical tumor??",
           "4": "What is Family Commons at St. Jude?"}

query_choice = input("Enter number of your query choice")
selected_query = QUERIES.get(query_choice)

rag_prompt = create_rag_prompt(selected_query,
                   db_retriever = db,
                   embedder = embedder,
                   k_docs = 10,
                   lambda_mult = 0.3,
                   query_prefix = 'query: ',
                   doc_prefix = 'passage: ',
                   use_query_expansion = True)

Enter number of your query choice 3


=== RAG Augmented Prompt ===

You are a helpful healthcare information assistant for families, patients, and caregivers.
You specialize in providing clear, accurate, and supportive information about childhood diseases,
cancer, and St. Jude Children’s Research Hospital.

When answering:
- Always rely on the retrieved documents (disease info pages, hospital resources, treatment guides).
- Present information in clear, compassionate language that non-experts can understand.
- Never provide personal medical advice, diagnoses, or treatment recommendations.
  Instead, encourage patients to consult their doctors for medical concerns.

Answer structure:
If relevant information is found:
- Summarize the key facts clearly and accurately.
- Organize the response with short paragraphs, bullet points, or sections if helpful.
- Cite or mention the source section in natural language (e.g., “According to St. Jude’s page on brain tumors…”).
- Keep the tone supportive and empathetic.
- Please provide an

In [173]:
!nvidia-smi

Mon Sep 29 20:47:11 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   72C    P0             32W /   70W |    8723MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [214]:

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(GEMMA_PATH)

messages = [
    {
        "role": "system",
        "content": [
            {"type": "text", "text": rag_prompt}
        ],
    },
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "What is an adrenocortical tumor?"}
        ],
    },
]

text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
inputs = tokenizer(text, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=512)

answer = tokenizer.decode(
    outputs[0][inputs["input_ids"].shape[1]:],
    skip_special_tokens=True
)
print(answer)

Here’s a breakdown of the information about adrenocortical tumors, based on the provided text:

**What is an Adrenocortical Tumor?**

An adrenocortical tumor is a rare tumor of the adrenal gland. It’s a type of cancer that can be benign (normal-looking) or malignant (cancerous).  It’s often referred to as an Adrenocortical cancer, or Adrenocortical adenoma.

**Key Points:**

*   **Location:** They can occur in the eye, head, neck, or urinary tract.
*   **Size:** The tumor size can vary, but it’s generally less than 5 cm.
*   **Lymph Nodes:** Cancer cells can spread to nearby lymph nodes.
*   **Metastasis:** The tumor can spread to other parts of the body.
*   **Treatment:** Treatment often involves chemotherapy and surgery.

**Important Note:** The text emphasizes that treatment is often a combination of therapies, including chemotherapy.

**Disclaimer:** *I am an AI Chatbot and not a medical professional. This information is for general knowledge and informational purposes only, and d

In [199]:

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(GEMMA_PATH)

messages = [
    {
        "role": "system",
        "content": [
            {"type": "text", "text": rag_prompt}
        ],
    },
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "What is Family Commons at St. Jude?"}
        ],
    },
]

text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
inputs = tokenizer(text, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=512)

answer = tokenizer.decode(
    outputs[0][inputs["input_ids"].shape[1]:],
    skip_special_tokens=True
)
print(answer)

Okay, here’s a breakdown of Family Commons at St. Jude, based on the provided text:

**Family Commons at St. Jude** is a treatment-free zone open only to patients and families. It’s a quiet, private space designed for relaxation between appointments, offering Resting Nooks for quiet time. It also provides a range of support services, including communication needs, child life, social work, spiritual care, and a concierge service.  St. Jude also arranges patient housing and support services, and offers creative spaces where patients and families can engage in artistic activities.

**Source:** St. Jude’s page on Family Commons.

Do you want me to elaborate on any of these points, or perhaps provide information on a specific aspect of Family Commons?wen খানiono


In [206]:

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(GEMMA_PATH)

messages = [
    {
        "role": "system",
        "content": [
            {"type": "text", "text": rag_prompt}
        ],
    },
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "What is adrenocortical tumor?"}
        ],
    },
]

text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
inputs = tokenizer(text, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=512)

answer = tokenizer.decode(
    outputs[0][inputs["input_ids"].shape[1]:],
    skip_special_tokens=True
)
print(answer)

Here’s a breakdown of the information about adrenocortical tumors, based on the provided text:

**What is an Adrenocortical Tumor (ACT)?**

An adrenocortical tumor is a rare tumor of the adrenal gland. It’s a type of cancer that can be benign (normal-looking) or malignant (cancerous).  It’s often referred to as an “Adrenocortical cancer” or “Adrenocortical adenoma.”

**Key Facts:**

*   **Location:** These tumors can occur in the eye, head, neck, or urinary tract.
*   **Size:** They can range in size from small (less than 5 cm) to larger than 5 cm.
*   **Lymph Nodes:**  ACTs can sometimes spread to nearby lymph nodes.
*   **Metastasis:**  The tumor can spread to other parts of the body.
*   **Treatment:** Treatment typically involves chemotherapy, surgery, and radiation therapy.

**Important Note:**  The text emphasizes that treatment often includes chemotherapy as part of the process.

**Disclaimer:** *I am an AI Chatbot and not a medical professional. This information is for general 

In [52]:
# # !pip install huggingface_hub
# # !huggingface-cli login

# from kaggle_secrets import UserSecretsClient
# user_secrets = UserSecretsClient()
# huggingface_token = user_secrets.get_secret("huggingface")

In [53]:
# !mkdir -p ~/.huggingface

In [54]:
# !echo -n $secret_value_0 > ~/.huggingface/token

In [55]:
# !huggingface-cli whoami

In [155]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM, pipeline
from langchain.llms import HuggingFacePipeline

MODELS = {"1": {"model_name":"google/flan-t5-base", "model_type": "encoder-decoder"},
          "2": {"model_name":"mistralai/Mistral-7B-Instruct-v0.3", "model_type": "decoder only"},
          "3": {"model_name":"meta-llama/Llama-2-7b-hf", "model_type": "decoder only"},
          "4": {"model_name":"google/gemma-3-4b-it", "model_type": "decoder only"}
         }

MODEL_CONFIG = {"encoder-decoder": {"model_cls": "text2text-generation",
                                    "pipeline": AutoModelForSeq2SeqLM},
                "decoder only": {"model_cls": "text-generation",
                                 "pipeline": AutoModelForCausalLM}}

In [156]:
def load_llm_model(selected_model: dict,
                   max_tokens: int = 1024):
    model_name = selected_model['model_name']
    model_type = selected_model['model_type']

    model_config_cls = MODEL_CONFIG[model_type]['model_cls']
    model_config_pipeline = MODEL_CONFIG[model_type]['pipeline']

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = model_config_pipeline.from_pretrained(model_name,
                                            device_map="auto",
                                            torch_dtype="auto")

    pipe = pipeline(model_config_cls,
                    model = model,
                    tokenizer = tokenizer,
                    max_new_tokens = max_tokens)
    llm = HuggingFacePipeline(pipeline = pipe)

    return llm

In [158]:
def main():
    choice = input("Enter number of your LLM model choice")
    selected_model = MODELS.get(choice)

    llm = load_llm_model(selected_model)

    query_choice = input("Enter number of your query choice: ")
    selected_query = QUERIES.get(query_choice)

    rag_prompt = create_rag_prompt(selected_query,
                   db_retriever = db,
                   embedder = embedder,
                   k_docs = 5,
                   lambda_mult = 0.3,
                   query_prefix = 'query: ',
                   doc_prefix = 'passage: ',
                   use_query_expansion = True
                   )

    response = llm(rag_prompt)
    print("\n=== Generated Answer ===")
    print(response)

## Question 1: What is adrenocortical tumor??

In [168]:
!pip install git+https://github.com/huggingface/transformers@v4.49.0-Gemma-3 

from transformers import AutoTokenizer
from transformers.models.gemma3 import Gemma3ForCausalLM

  Cloning https://github.com/huggingface/transformers (to revision v4.49.0-Gemma-3) to /tmp/pip-req-build-30je5c99
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-30je5c99
  Running command git checkout -q 1c0f782fe5f983727ff245c4c1b3906f9b99eec2
  Resolved https://github.com/huggingface/transformers to commit 1c0f782fe5f983727ff245c4c1b3906f9b99eec2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.50.0.dev0-py3-none-any.whl size=10936457 sha256=033dc13cec250688e99fc80728565dbbbea580a29c001374c87be8ad8b1e7b33
  Stored in directory: /tmp/pip-ephem-wheel-cache-23kvikz9/wheels/53/15/d5/d63b866c641d8863f9cd29a4cc7a5efc38476c3aae8247c195
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.52.4
    Uninstalling transf

In [169]:
GEMMA_PATH = "/kaggle/input/gemma-3/transformers/gemma-3-1b-it/1"
processor = AutoTokenizer.from_pretrained(GEMMA_PATH)
model = Gemma3ForCausalLM.from_pretrained(GEMMA_PATH)

In [ ]:

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(GEMMA_PATH)

messages = [
    {
        "role": "system",
        "content": [
            {"type": "text", "text": prompt}
        ],
    },
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "What is an adrenocortical tumor?"}
        ],
    },
]

text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
inputs = tokenizer(text, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=512)

answer = tokenizer.decode(
    outputs[0][inputs["input_ids"].shape[1]:],
    skip_special_tokens=True
)
print(answer)


In [160]:
import torch
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()

In [164]:
import gc, torch
# Delete big objects first (models, tensors, dataloaders, outputs, loss)
for name in ["model","optimizer","scheduler","outputs","loss","batch","embeddings", "llm"]:
    if name in globals(): del globals()[name]

gc.collect()                 # free Python refs
torch.cuda.empty_cache()     # release cached blocks to the driver
torch.cuda.ipc_collect()     # clean inter-proc handles
print(torch.cuda.memory_summary())

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 2            |        cudaMalloc retries: 2         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |  97371 KiB |   6779 MiB |    832 GiB |    832 GiB |
|       from large pool |  82071 KiB |   6748 MiB |    814 GiB |    814 GiB |
|       from small pool |  15300 KiB |     31 MiB |     17 GiB |     17 GiB |
|---------------------------------------------------------------------------|
| Active memory         |  97371 KiB |   6779 MiB |    832 GiB |    832 GiB |
|       from large pool |  82071 KiB |   6748 MiB |    814 GiB |

In [165]:
!nvidia-smi

Mon Sep 29 20:29:58 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   73C    P0             33W /   70W |    4905MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [167]:
if __name__ == "__main__":
    main()

Enter number of your LLM model choice 4


tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

...

Device set to use cuda:0


Enter number of your query choice:  3


=== RAG Augmented Prompt ===

You are a helpful healthcare information assistant for families, patients, and caregivers.
You specialize in providing clear, accurate, and supportive information about childhood diseases,
cancer, and St. Jude Children’s Research Hospital.

When answering:
- Always rely on the retrieved documents (disease info pages, hospital resources, treatment guides).
- Present information in clear, compassionate language that non-experts can understand.
- Never provide personal medical advice, diagnoses, or treatment recommendations.
  Instead, encourage patients to consult their doctors for medical concerns.

Answer structure:
1. If relevant information is found:
   - Summarize the key facts clearly and accurately.
   - Organize the response with short paragraphs, bullet points, or sections if helpful.
   - Cite or mention the source section in natural language (e.g., “According to St. Jude’s page on brain tumors…”).
   - Keep the tone supportive and empathetic.

2. 

W0929 20:33:03.763000 36 torch/_inductor/utils.py:1137] [0/0] Not enough SMs to use max_autotune_gemm mode
/usr/local/lib/python3.11/dist-packages/torch/_inductor/compile_fx.py:1948: UserWarning: Tesla T4 does not support bfloat16 compilation natively, skipping
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/_inductor/compile_fx.py:1948: UserWarning: Tesla T4 does not support bfloat16 compilation natively, skipping
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/_inductor/compile_fx.py:1948: UserWarning: Tesla T4 does not support bfloat16 compilation natively, skipping
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/_inductor/compile_fx.py:1948: UserWarning: Tesla T4 does not support bfloat16 compilation natively, skipping
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/_inductor/compile_fx.py:1948: UserWarning: Tesla T4 does not support bfloat16 compilation natively, skipping
  warnings.warn(
/usr/local/lib/python3.11/dist-pa

Unsupported: generator

## Question 2: What are the admission requirements for the MS in Applied Data Science program?

In [74]:
!pip -q install gpustat
!gpustat -p
!fuser -v /dev/nvidia* 2>/dev/null
!ps -eo pid,user,comm,args --sort=-%mem | grep -E "python|torchrun|accelerate|ray|jax|tensorflow"
# If you see your own PIDs:
#!kill -9 <PID1> <PID2>

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.1/98.1 kB 3.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.1/85.1 kB 6.8 MB/s eta 0:00:00
780e08210b09  Mon Sep 29 20:08:55 2025  560.35.03
[0] Tesla T4 | 72°C,   0 % | 11578 / 15360 MB |
[1] Tesla T4 | 70°C,   0 % |  7498 / 15360 MB |
    36    39    36    39    36    39    36    39     36 root     python3         /usr/bin/python3 -m colab_kernel_launcher -f /root/.local/share/jupyter/runtime/kernel-59acc857-d0de-4523-9679-3b1d5eecf622.json
    127 root     python3         /usr/bin/python3 -m pylsp
      1 root     jupyter-noteboo /usr/bin/python3 /usr/local/bin/jupyter-notebook --ip=* --NotebookApp.base_url=/k/264742343/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2IiwidHlwIjoiSldUIn0..nxRh_6aIueadeLxIU4IHAw.wHn2usmsQSXh_xW90mm6ls8Ke4PRhQV95gd5YmCpCXG55LDJoCxcGq7Bh3HaxiuiRox0_YwasSA_nIj

In [72]:
import gc, torch
# delete big objects you created (models, tensors, dataloaders, etc.)
for n in ["model","optimizer","scheduler","outputs","loss","batch","embeddings"]:
    if n in globals(): del globals()[n]
gc.collect()
torch.cuda.empty_cache()
torch.cuda.ipc_collect()      # extra cleanup
print(torch.cuda.memory_summary(device=None, abbreviated=True))

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 2            |        cudaMalloc retries: 2         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |   6692 MiB |  14706 MiB | 770293 MiB | 763600 MiB |
|---------------------------------------------------------------------------|
| Active memory         |   6692 MiB |  14706 MiB | 770293 MiB | 763600 MiB |
|---------------------------------------------------------------------------|
| Requested memory      |   6691 MiB |  14704 MiB | 768602 MiB | 761910 MiB |
|---------------------------------------------------------------

In [76]:
!nvidia-smi

Mon Sep 29 20:09:41 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   72C    P0             32W /   70W |   11579MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [69]:
if __name__ == "__main__":
    main()

Enter number of your LLM model choice 2


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0


Enter number of your query choice:  3


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


=== RAG Augmented Prompt ===

You are a helpful healthcare information assistant for families, patients, and caregivers.
You specialize in providing clear, accurate, and supportive information about childhood diseases,
cancer, and St. Jude Children’s Research Hospital.

When answering:
- Always rely on the retrieved documents (disease info pages, hospital resources, treatment guides).
- Present information in clear, compassionate language that non-experts can understand.
- Never provide personal medical advice, diagnoses, or treatment recommendations.
  Instead, encourage patients to consult their doctors for medical concerns.

Answer structure:
1. If relevant information is found:
   - Summarize the key facts clearly and accurately.
   - Organize the response with short paragraphs, bullet points, or sections if helpful.
   - Cite or mention the source section in natural language (e.g., “According to St. Jude’s page on brain tumors…”).
   - Keep the tone supportive and empathetic.

2. 

OutOfMemoryError: CUDA out of memory. Tried to allocate 472.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 148.12 MiB is free. Process 3015 has 14.59 GiB memory in use. Of the allocated memory 14.36 GiB is allocated by PyTorch, and 107.05 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

## Question 3: Can you provide information about the capstone project??

In [ ]:
if __name__ == "__main__":
    main()

Enter number of your LLM model choice 2


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.
Device set to use cuda:0


Enter number of your query choice:  3


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


=== RAG Augmented Prompt ===

    You are an educational assistant. 
    Your task is to extract information related to Master of Science in Applied Data Science program, such as admission requirement, curriculum, faculties, career outcomes, and etc.
    Use the following context to answer the question.

    [CONTEXT]
    passage: .Data Science Capstone ProjectThe required Capstone Project is completed over two quarters and covers research design, implementation, and writing.Full-time students start their capstone project in their third quarter. Part-time students generally begin the capstone project in their fifth quarter.
passage: . A list of sample electives (subject to change) appear at the bottom of theIn-PersonandOnline Programpages.CapstoneData Science Capstone ProjectLetter GradeThe required Capstone Project is completed over two quarters and covers research design, implementation, and writing.Full-time students start their capstone project in their third quarter. Part-time stu